In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [2]:
dataset = pd.read_csv('data.csv')
# dataset

In [3]:
X = dataset.drop('xAttack', axis = 1)
Y = dataset['xAttack']

In [4]:
X = (X - np.mean(X))/np.std(X)
X.shape

(125973, 29)

In [5]:
X = X.iloc[:]
# X_copy = copy.deepcopy(X_train)
# Y_copy = copy.deepcopy(Y_train)
# Y_train
Y=pd.concat([Y,pd.get_dummies(Y,prefix='xAttack')],axis=1).drop(['xAttack'],axis=1)
Y = Y.iloc[:]
# X

In [6]:
class Neural_network(object):
    
    def __init__(self):
        self.input_layer_size = 29
        self.output_layer_size = 29
        self.hidden_layer_size = 14
        self.hidden_layers = 1
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.epoch = 50
        self.ita = 0.001
        self.a = {}
        self.Y_hat = np.array([])
        self.delta = {}
        
    def initialization(self):
        
        self.weights[1] = np.random.randn(self.input_layer_size,self.hidden_layer_size)*0.01
        self.weights[self.hidden_layers+1] = np.random.randn(self.hidden_layer_size,self.output_layer_size)*0.01
        
        for i in np.arange(2,self.hidden_layers+1):
            self.weights[i] = np.random.randn(self.hidden_layer_size,self.hidden_layer_size)*0.01
            
    
    def softmax(self,a):
        tsum = np.sum(a,axis = 1,keepdims = True)
        return np.divide(a,tsum)
    
    
    def error(self,y):
#         return sum(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))/y.shape[0]
        return np.mean(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))
    
    def derivative_sigmoid(self,layer):
        derivative = layer*(1-layer)
        return derivative
    
   
    def deltas(self,y):
        w = self.a[self.hidden_layers+2]
        error = (y - self.Y_hat)
        self.delta[self.hidden_layers+2] = -1 * error * self.derivative_sigmoid(w)
        xx = self.a[self.hidden_layers+1].T
        yy = self.delta[self.hidden_layers+2]
        self.dweights[self.hidden_layers+1] = np.dot(xx , yy)
        
        for i in np.arange(self.hidden_layers+1,1,-1):
            aa = self.delta[i+1]
            bb = self.weights[i].T
            deri = self.derivative_sigmoid(self.a[i])
            self.delta[i] = ( np.dot(aa, bb))*deri
            self.dweights[i-1] = np.dot(self.a[i-1].T , (self.delta[i]))  
    
    def sigmoid(self,x):
        x = -x
        return 1 / (1 + np.exp(x))
    
    
    
    def forward_propagation(self, X):
        self.a[1]=np.array(X)
        
        fg = 1
        for i in range(1,self.hidden_layers+2):
            if fg:
                self.Z[i+1] = np.dot(self.a[i],self.weights[i])
                self.a[i+1] = self.sigmoid(self.Z[i+1])
        self.Y_hat = self.softmax(self.a[self.hidden_layers+2])
        
   
    def gradient_descent(self,X_train,Y_train):

        k = 0
        while k <= 120000:
            for i in range(self.epoch):
                Y = Y_train[k:k+5000]
                X = X_train[k:k+5000]
                self.forward_propagation(X)
                self.deltas(Y)
                for j in range(1,self.hidden_layers+2):
                    lrweights = self.ita*self.dweights[j]
                    self.weights[j] = self.weights[j] - lrweights
            print("Final Error : ",self.error(Y))
            k = k + 5000

        
    def predict(self,X):
        self.a[1] = np.array(X)
        self.Z[1] = np.dot(self.a[1],self.weights[1])
        return self.sigmoid(self.Z[1])
    
            

In [7]:
k = Neural_network()
k.initialization()
k.gradient_descent(X,X)

Final Error :  11.543475273528047
Final Error :  11.153243593581847
Final Error :  11.660679104086803
Final Error :  11.788187463304343
Final Error :  13.252736784305768
Final Error :  11.896893289071928
Final Error :  11.947305241220286
Final Error :  12.014548412644285
Final Error :  16.65997267473713
Final Error :  11.709874174850292
Final Error :  10.252503065924468
Final Error :  29.54103856479301
Final Error :  24.13184546525936
Final Error :  12.761460622971182
Final Error :  11.02372107634145
Final Error :  11.449614104815756
Final Error :  12.572095667551391
Final Error :  11.57937808597775
Final Error :  12.611228248181297
Final Error :  11.27714419058316
Final Error :  12.131411879543819
Final Error :  10.924499663947278
Final Error :  11.880091423204963
Final Error :  11.250989053109985
Final Error :  12.453711905508358


In [8]:
# X_test = X_test[0:1000]
# Y_test = Y_test[0:1000]

finaldata=k.predict(X)
finaldata=pd.DataFrame(finaldata)
df_final = pd.concat([finaldata,dataset['xAttack']],axis=1)
# df_final

In [9]:
df_final.to_csv("data_final.csv",index=False)